<a href="https://colab.research.google.com/github/henrhie/A_star/blob/master/nb/Qwen2.5_(3B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth-zoo==2025.7.4
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
lora_rank      = 64

# 1. Load a clean “System 1” model (no LoRA)
sys1, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length           = max_seq_length,
    load_in_4bit             = True,
    fast_inference           = True,
    gpu_memory_utilization   = 0.5,
)

# Freeze its parameters so no gradients flow here
for p in sys1.parameters():
    p.requires_grad_(False)
sys1.eval()


Unsloth: Please update Unsloth and Unsloth-Zoo to the latest version!
Do this via `pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo`
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-21 06:52:03 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-21 06:52:04 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.5: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-1.5b-instruct-

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
    (layers): ModuleList(
      (0): Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )


In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [6]:
# 2. Load a second, separate model for “System 2”
sys2, _ = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B-Instruct",  # same checkpoint
    max_seq_length           = max_seq_length,
    load_in_4bit             = True, # Load without 4bit
    fast_inference           = False, # Load without fast_inference
    gpu_memory_utilization   = 0.5, # This is not needed for the second model loaded this way
)

# 3. Wrap System 2 in LoRA adapters
sys2 = FastLanguageModel.get_peft_model(
    sys2,
    r                       = lora_rank,
    target_modules         = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha             = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state           = 3407,
)

==((====))==  Unsloth 2025.7.5: Fast Qwen2 patching. Transformers: 4.53.2. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.7.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [66]:
SYS_1_PROMPT = """
You are System 1, the final answer engine.
Your sole job is to read the original question plus the provided chain-of-thought reasoning, then output the **concise, correct final answer**.
Follow these rules:

1. **Role & Tone**
   - You are concise and authoritative.
   - Provide only the final answer—no reasoning steps or commentary.

2. **Input Format**
   Every input will be structured as:
     ```
     Question: ...
     Reasoning: ...
     ```
   - You must respect the reasoning; do not ignore it or re-solve from scratch.

3. **Output Format**
   - Directly write the answer after the “Answer:” cue.
   - If a numeric answer, write only the number (with units if asked).
   - If a textual answer, write only the text.
   - Do not include “Question:” or “Reasoning:” in your output.
   - Ensure that your final answer is placed in answer tags (</answer></answer>)

4. **Verification**
   - If the reasoning contradicts itself or is unclear, choose the most logical interpretation.
"""

In [59]:
import re
from datasets import load_dataset, Dataset

SYS_2_PROMPT = """
You are System 2, the deliberate reasoning engine.
Your sole job is to think **step by step** and produce a **chain of thought** (CoT)
that leads to the solution—**do not** ever state or output the final answer itself.
When given a question or prompt, follow these rules:

1. **Role & Tone**
   - You are thorough, precise, and logical.
   - Use clear, numbered or bullet-style steps.
   - Do NOT include any final answer or conclusion in your output.

2. **Supported Query Types**
   - **Math/Logic:** show all intermediate calculations.
   - **Commonsense/Narrative:** articulate each inference.
   - **Code-related:** explain algorithm design, data structures, and line-by-line logic.
   - **General knowledge:** cite relevant facts and sources as needed.

3. **Format**
   Begin every response exactly as:
     ```
     Question: {original question}
     Let's think step by step:
     1. …
     2. …
     3. …
     ```
   - Each step should be on its own line, numbered or bullet-pointed.
   - End your output when the reasoning is complete, but **stop before giving the final answer**.

4. **Cheat Prevention**
   - Do **not** include phrases like “therefore the answer is …” or reveal the numeric/textual answer.
   - If you accidentally deduce the answer, rephrase as an inference step only (e.g. “This leads us to identify the key value”).

5. **Length & Clarity**
   - Keep maximum tokens to your configured limit (e.g., 128 tokens).
   - Be as concise as possible while still covering all necessary reasoning.

6. **Example**
   **Prompt:** “If 3x + 5 = 20, what is x?”
   **Your output:**
"""


def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def extract_cot(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[0].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split]
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYS_2_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer']),
        'cot': extract_cot(x['answer'])
    })
    return data

dataset = get_gsm8k_questions()

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [81]:
#run inference on sys1 model with question and cot
text1 = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYS_1_PROMPT},
    {"role" : "user", "content" : 'Question: '+ dataset[5]['question'] + '\n' + 'Reasoning: '+ dataset[5]['cot']},
], tokenize = False, add_generation_prompt = True)
text2 = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYS_1_PROMPT},
    {"role" : "user", "content" : 'Question: '+ dataset[5]['question'] + '\n' + 'Reasoning: '+ dataset[5]['cot']},
], tokenize = False, add_generation_prompt = True)
input = tokenizer([text1, text2], return_tensors = "pt")
output = sys1.generate(
    input_ids      = input["input_ids"].to(sys1.device),
    attention_mask = input["attention_mask"].to(sys1.device))
print(tokenizer.batch_decode(output)[0])

<|im_start|>system

You are System 1, the final answer engine.  
Your sole job is to read the original question plus the provided chain-of-thought reasoning, then output the **concise, correct final answer**.  
Follow these rules:

1. **Role & Tone**  
   - You are concise and authoritative.  
   - Provide only the final answer—no reasoning steps or commentary.

2. **Input Format**  
   Every input will be structured as:
     ```
     Question: ...
     Reasoning: ...
     ```
   - You must respect the reasoning; do not ignore it or re-solve from scratch.

3. **Output Format**  
   - Directly write the answer after the “Answer:” cue.  
   - If a numeric answer, write only the number (with units if asked).  
   - If a textual answer, write only the text.  
   - Do not include “Question:” or “Reasoning:” in your output.
   - Ensure that your final answer is placed in answer tags (</answer></answer>)

4. **Verification**  
   - If the reasoning contradicts itself or is unclear, choose the

In [52]:
# #run inference on dataset[0]
# text = tokenizer.apply_chat_template([
#     {"role" : "system", "content" : SYS_2_PROMPT},
#     {"role" : "user", "content" : dataset[5]['question']},
# ], tokenize = False, add_generation_prompt = True)
# input = tokenizer(text, return_tensors = "pt")
# output = sys2.generate(
#     input_ids      = input["input_ids"].to(sys2.device),
#     attention_mask = input["attention_mask"].to(sys2.device))
# print(tokenizer.decode(output[0]))

In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def sys_1_reward(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    inputs = [
          tokenizer.apply_chat_template([
          {"role" : "system", "content" : SYS_1_PROMPT},
          {"role" : "user", "content" : 'Question: ' + q + '\n' + 'Reasoning: ' + r + '\n Answer: '},
      ], tokenize = False, add_generation_prompt = True)
        for r in responses
    ]

    print('input111', inputs[0])

    batch = tokenizer(
      inputs,
      return_tensors="pt",
      padding=True,
      truncation=True,
    ).to(device)
    out_ids = sys1.generate(
      input_ids      = batch["input_ids"],
      attention_mask = batch["attention_mask"],
      max_new_tokens = 32,
      do_sample      = True,
      pad_token_id   = tokenizer.eos_token_id,
    )
    raw_outputs = tokenizer.batch_decode(out_ids, skip_special_tokens=True)

    print("answer---->: ", answer[0])
    extracted_responses = [extract_xml_answer(r) for r in raw_outputs]
    print("outputs---->: ", extracted_responses[0])
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [71]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    # use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


In [72]:
trainer = GRPOTrainer(
    model = sys2,
    processing_class = tokenizer,
    reward_funcs = [
        sys_1_reward
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 73,859,072 of 1,617,573,376 (4.57% trained)


input111 <|im_start|>system

You are System 1, the final answer engine.  
Your sole job is to read the original question plus the provided chain-of-thought reasoning, then output the **concise, correct final answer**.  
Follow these rules:

1. **Role & Tone**  
   - You are concise and authoritative.  
   - Provide only the final answer—no reasoning steps or commentary.

2. **Input Format**  
   Every input will be structured as:
     ```
     Question: ...
     Reasoning: ...
     ```
   - You must respect the reasoning; do not ignore it or re-solve from scratch.

3. **Output Format**  
   - Directly write the answer after the “Answer:” cue.  
   - If a numeric answer, write only the number (with units if asked).  
   - If a textual answer, write only the text.  
   - Do not include “Question:” or “Reasoning:” in your output.
   - Ensure that your final answer is placed in answer tags (</answer></answer>)

4. **Verification**  
   - If the reasoning contradicts itself or is unclear, c

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / sys_1_reward / mean,rewards / sys_1_reward / std
1,0.000000,0.000000,0.000000,196.000000,180.000000,200.000000,0.750000,184.000000,180.000000,188.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,199.500000,196.000000,200.000000,0.875000,196.000000,196.000000,196.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,192.500000,140.000000,200.000000,0.875000,140.000000,140.000000,140.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,171.875000,112.000000,200.000000,0.500000,143.750000,112.000000,192.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,127.000000,89.000000,200.000000,0.250000,102.666672,89.000000,115.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,182.125000,123.000000,200.000000,0.500000,164.250000,123.000000,191.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,200.000000,200.000000,200.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,166.125000,101.000000,200.000000,0.375000,145.800003,101.000000,184.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,105.625000,12.000000,200.000000,0.250000,74.166672,12.000000,157.000000,0.000000,0.000000,0.000000


input111 <|im_start|>system

You are System 1, the final answer engine.  
Your sole job is to read the original question plus the provided chain-of-thought reasoning, then output the **concise, correct final answer**.  
Follow these rules:

1. **Role & Tone**  
   - You are concise and authoritative.  
   - Provide only the final answer—no reasoning steps or commentary.

2. **Input Format**  
   Every input will be structured as:
     ```
     Question: ...
     Reasoning: ...
     ```
   - You must respect the reasoning; do not ignore it or re-solve from scratch.

3. **Output Format**  
   - Directly write the answer after the “Answer:” cue.  
   - If a numeric answer, write only the number (with units if asked).  
   - If a textual answer, write only the text.  
   - Do not include “Question:” or “Reasoning:” in your output.
   - Ensure that your final answer is placed in answer tags (</answer></answer>)

4. **Verification**  
   - If the reasoning contradicts itself or is unclear, c

KeyboardInterrupt: 

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s, est. speed input: 63.38 toks/s, output: 25.69 toks/s]


'There are 2 r\'s in the word "strawberry."'

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.06s/it, est. speed input: 14.05 toks/s, output: 29.09 toks/s]


'<reasoning>\nTo find out how many times the letter \'r\' appears in the word "strawberry", we can go through the word character by character and count each occurrence of \'r\'. In "strawberry", the letter \'r\' appears 3 times: once in the beginning, once in the middle, and once at the end of the word.\n</reasoning>\n<answer>\n3\n</answer>'

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
